## kickoff_for_each

Manual triage is slow and inconsistent — a human support manager has to read every ticket, decide its priority, route it to the right team, and draft a response. With hundreds of tickets coming in daily this doesn't scale and different people classify the same issue differently.

Each ticket needs the same pipeline — every ticket regardless of content goes through the same two steps: classify first, then resolve. This is a perfect pattern for automation because the workflow never changes, only the input data changes.
Tickets are independent of each other — Alice's payment issue has zero relation to Bob's seat request. So they can be processed through the same crew without any shared state or dependency, which is exactly what kickoff_for_each handles cleanly.
Without this approach your alternatives would be:

Manually looping and calling kickoff() yourself, managing errors per iteration
Processing tickets one at a time with no structure
Building a custom orchestration layer just to repeat the same pipeline

So in one line — it automates a structured, repeatable multi-step workflow across an arbitrary number of independent inputs, which is one of the most common real world problems in business operations, whether it's support tickets, invoices, resumes, contracts, or any document that needs consistent processing at scale.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from crewai import Crew,Agent,Task,LLMs

In [3]:
llm = LLM(
    model="ollama/deepseek-v3.1:671b-cloud",
    base_url="http://localhost:11434",
    temperature=0.2
)

In [4]:
# Agents
classifier_agent = Agent(
    role="Ticket Classifier",
    goal="Classify support tickets by priority and category",
    backstory="You are an experienced support manager who triages customer issues.",
    llm = llm
)

resolver_agent = Agent(
    role="Support Resolver",
    goal="Draft resolution responses for support tickets",
    backstory="You are a senior support engineer who resolves customer issues efficiently.",
    llm=llm
)


In [5]:
# Tasks
classify_task = Task(
    description="""
        Analyze this support ticket and classify it:
        Customer: {customer_name}
        Issue: {issue_description}
        Plan: {subscription_plan}
        
        Classify by:
        - Priority (Critical / High / Medium / Low)
        - Category (Billing / Technical / Account / Feature Request)
    """,
    agent=classifier_agent,
    expected_output="Priority and category classification with reasoning"
)

resolve_task = Task(
    description="""
        Based on the classification, draft a professional response for:
        Customer: {customer_name}
        Issue: {issue_description}
        
        Response should include resolution steps and estimated time to fix.
    """,
    agent=resolver_agent,
    expected_output="A complete customer response email with resolution steps"
)



In [6]:
# Crew
support_crew = Crew(
    agents=[classifier_agent, resolver_agent],
    tasks=[classify_task, resolve_task],
    verbose=True
)

In [ ]:
# Real world tickets coming in from a database or queue
tickets = [
    {
        "customer_name": "Alice Johnson",
        "issue_description": "Unable to process payments, getting gateway timeout error since 2 hours",
        "subscription_plan": "Enterprise"
    },
    {
        "customer_name": "Bob Smith",
        "issue_description": "Need to add 5 more seats to my account",
        "subscription_plan": "Business"
    },
    {
        "customer_name": "Carol White",
        "issue_description": "Dashboard charts are not loading, blank screen after login",
        "subscription_plan": "Starter"
    },
    {
        "customer_name": "David Lee",
        "issue_description": "Wrongly charged twice for this month subscription",
        "subscription_plan": "Pro"
    }
]

# Process every ticket independently through the same crew
results = support_crew.kickoff_for_each(inputs=tickets)


╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  38a6f919-2de1-4b4e-a3b5-c1768ff820d3                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name:                                                                                                          │
│          Analyze this support ticket and classify it:                                                           │
│          Customer: Alice Johnson                                                                                │
│          Issue: Unable to process payments, getting gateway timeout error since 2 hours                         │
│          Plan: Enterprise                                                                                       │
│                                                                                                                 │
│          Classify by:                                                                                           │
│          - Priority (Critical / High / Medium / Low)                                                            │
│          - Category (Billing / Technical / Account / Feature Request)                                           │
│                                                                                                                 │
│  ID: 17a7d08a-f054-4397-a878-efaba88bf326                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Ticket Classifier                                                                                       │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Analyze this support ticket and classify it:                                                           │
│          Customer: Alice Johnson                                                                                │
│          Issue: Unable to process payments, getting gateway timeout error since 2 hours                         │
│          Plan: Enterprise                                                                                       │
│                                                                                                                 │
│          Classify by:                                                                                           │
│          - Priority (Critical / High / Medium / Low)                                                            │
│          - Category (Billing / Technical / Account / Feature Request)                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Ticket Classifier                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Priority:** Critical                                                                                         │
│  **Category:** Technical                                                                                        │
│                                                                                                                 │
│  **Reasoning:**                                                                                                 │
│  - **Priority:** The issue involves a complete inability to process payments, which directly impacts the        │
│  customer's revenue and business operations. The problem has persisted for 2 hours, indicating an ongoing       │
│  service disruption. For an Enterprise plan customer, this qualifies as a Critical priority due to the          │
│  immediate financial and operational impact.                                                                    │
│  - **Category:** The root cause is a gateway timeout error, which is a technical infrastructure or integration  │
│  issue rather than a billing discrepancy, account access problem, or feature enhancement request.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│                                                                                                                 │
│          Analyze this support ticket and classify it:                                                           │
│          Customer: Alice Johnson                                                                                │
│          Issue: Unable to process payments, getting gateway timeout error since 2 hours                         │
│          Plan: Enterprise                                                                                       │
│                                                                                                                 │
│          Classify by:                                                                                           │
│          - Priority (Critical / High / Medium / Low)                                                            │
│          - Category (Billing / Technical / Account / Feature Request)                                           │
│                                                                                                                 │
│  Agent:                                                                                                         │
│  Ticket Classifier                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Ticket for Alice Johnson:
Subject: Urgent: Resolution Steps for Payment Gateway Timeout Error

Dear Alice Johnson,

Thank you for reporting the issue you are experiencing with payment processing. We understand that receiving a gateway timeout error for the past two hours is critically impacting your business operations, and we sincerely apologize for the disruption this is causing.

Our engineering team was immediately notified and has identified a intermittent latency issue with one of our primary payment processing endpoints. This is causing the timeout errors you are encountering.

**Resolution Steps in Progress:**

A full resolution is currently being implemented by our infrastructure team. The steps are as follows:
1.  **Traffic Re-routing:** We are immediately shifting payment traffic to our redundant, high-availability endpoint to restore service.
2.  **Root Cause Mitigation:** Our engineers are applying a patch to the affected server to resolve the latency spike.
3.  **System M

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 



┌───────────────────────── Tracing Preference Saved ──────────────────────────┐
│                                                                             │
│  Info: Tracing has been disabled.                                           │
│                                                                             │
│  Your preference has been saved. Future Crew/Flow executions will not       │
│  collect traces.                                                            │
│                                                                             │
│  To enable tracing later, do any one of these:                              │
│  • Set tracing=True in your Crew/Flow code                                  │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file              │
│  • Run: crewai traces enable                                                │
│                                                                             │
└─────────────────────────────────────

In [13]:
print(type(results))

<class 'list'>


In [11]:
print(tickets)

[{'customer_name': 'Alice Johnson', 'issue_description': 'Unable to process payments, getting gateway timeout error since 2 hours', 'subscription_plan': 'Enterprise'}, {'customer_name': 'Bob Smith', 'issue_description': 'Need to add 5 more seats to my account', 'subscription_plan': 'Business'}, {'customer_name': 'Carol White', 'issue_description': 'Dashboard charts are not loading, blank screen after login', 'subscription_plan': 'Starter'}, {'customer_name': 'David Lee', 'issue_description': 'Wrongly charged twice for this month subscription', 'subscription_plan': 'Pro'}]


In [9]:
for ticket, result in zip(tickets, results):
    print(f"Ticket for {ticket['customer_name']}:")
    print(result)
    print("---")

Ticket for Alice Johnson:
Subject: Urgent: Resolution Steps for Payment Gateway Timeout Error

Dear Alice Johnson,

Thank you for reporting the issue you are experiencing with payment processing. We understand that receiving a gateway timeout error for the past two hours is critically impacting your business operations, and we sincerely apologize for the disruption this is causing.

Our engineering team was immediately notified and has identified a intermittent latency issue with one of our primary payment processing endpoints. This is causing the timeout errors you are encountering.

**Resolution Steps in Progress:**

A full resolution is currently being implemented by our infrastructure team. The steps are as follows:
1.  **Traffic Re-routing:** We are immediately shifting payment traffic to our redundant, high-availability endpoint to restore service.
2.  **Root Cause Mitigation:** Our engineers are applying a patch to the affected server to resolve the latency spike.
3.  **System M

In [14]:
names  = ["Alice", "Bob", "Carol"]
ages   = [25, 30, 35]
cities = ["NY", "LA", "SF"]

for name, age, city in zip(names, ages, cities):
    print(name, age, city)

# Output:
# Alice 25 NY
# Bob 30 LA
# Carol 35 SF

Alice 25 NY
Bob 30 LA
Carol 35 SF
